# Configuration

Funflow provides support for configuring a `Flow` via a YAML config file or environment variables. Support for automatically generated CLI flags is also planned but as of the writing of this tutorial has not been implemented.

If a Task you are using contains a `Configurable` field, you will need to specify the source for that configuration when you write your `Flow` via one of the `Literal`, `ConfigFromEnv`, or `ConfigFromFile` constructors. For example, the `Arg` field of the `DockerTask` supports configurable args. Let's take a look at a few examples.

## Environment Variables

In [1]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE GADTs #-}
-- Note: Using OverloadedStrings with DockerTask since it will automatically
-- make sure that any `Literal` strings we write are of type `Arg`
{-# LANGUAGE OverloadedStrings #-}

import Data.Maybe (isJust, isNothing)
import Path (Abs, Dir, File, Path, parseAbsDir, reldir, relfile, toFilePath, (</>))
import System.Directory (getCurrentDirectory)
import System.Environment (lookupEnv, setEnv, unsetEnv)

import qualified Data.CAS.ContentStore as CS
import Funflow
import Funflow.Tasks.Docker
import Funflow.Config (Configurable (Literal, ConfigFromFile, ConfigFromEnv, ConfigFromCLI))

flow1 = dockerFlow $ 
    DockerTaskConfig {
        image="alpine:latest",
        command="echo",
        args=["this is a hard-coded literal value, the next value is:", Arg $ ConfigFromEnv "CONFIGURING_FLOWS"]
    }

Now we just need to set the `CONFIGURING_FLOWS` environment variable and run the task

In [2]:
setEnv "CONFIGURING_FLOWS" "'hello from an environment variable!'"

runFlow flow1 DockerTaskInput {inputBindings = [], argsVals = mempty} :: IO (CS.Item)

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T03:38:07.758648984Z this is a hard-coded literal value, the next value is: hello from an environment variable!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## YAML File
For passing in configurations via a config file, use the `ConfigFromFile` constructor and pass a config file path to `runFlowWithConfig`.

In [3]:
{-# LANGUAGE QuasiQuotes #-}

getStoreAndConfig :: IO (Path Abs Dir, Path Abs File)
getStoreAndConfig = do
    cwd <- parseAbsDir =<< getCurrentDirectory
    let storeDirPath = cwd </> [reldir|./.tmp/store|]
        configFilePath = cwd </> [relfile|./flow.yaml|]
    return (storeDirPath, configFilePath)

In [4]:
-- Inspect the config file.
lines <$> ((toFilePath . snd <$> getStoreAndConfig) >>= readFile)

["ourMessage: \"Hello from the flow.yaml\"","ourOtherValue: 42"]

In [5]:
goodFileArg = Arg $ ConfigFromFile "ourMessage"

runWithEmptyInput :: [Arg] -> IO CS.Item
runWithEmptyInput confArgs = 
    let flowConf = DockerTaskConfig{ image = "alpine:latest", command = "echo", args = confArgs }
    in do
        (storeDir, confFile) <- getStoreAndConfig
        let runConf = RunFlowConfig{ configFile = Just confFile, storePath = storeDir }
        runFlowWithConfig runConf (dockerFlow flowConf) DockerTaskInput{ inputBindings = [], argsVals = mempty }

In [6]:
runWithEmptyInput [goodFileArg]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T03:38:13.016307800Z Hello from the flow.yaml
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

## Safety: _configuration_ is distinct from _execution_.

First, note that we can also __mix argument types__:

In [7]:
-- Mixing literal and file config
runWithEmptyInput ["GREETING 1", goodFileArg]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T03:38:15.996789848Z GREETING 1 Hello from the flow.yaml
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

In [8]:
setEnv "SECOND_GREETING" "checkItOut!"
lookupEnv "SECOND_GREETING"

Just "checkItOut!"

In [9]:
-- Mixing config file and env var.
runWithEmptyInput [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING"]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T03:38:19.174480420Z Hello from the flow.yaml checkItOut!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}

More critical, though, is that _configuration precedes execution_. This means that we can _avoid destined failure_ by detecting the it during interpretation, before we ever actually run anything.
Here we set up a simple example to demonstrate, but imagine the time (and perhaps monetary!) cost that could be saved when the destined failure would occur after a long-running computation.

In [10]:
-- Set stage for config-time error.
unsetEnv "SECOND_GREETING"
isNothing <$> lookupEnv "SECOND_GREETING"

True

In [11]:
-- Trigger a config-time error.
import Control.Exception.Safe (StringException(..), try)

do
    res <- try (runWithEmptyInput [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING"])
    case res of 
        Left (StringException msg _) -> putStrLn ("Caught error: " ++ msg)
        Right _ -> error "Unexpected success!"

Caught error: Missing the following required config keys: ["SECOND_GREETING"]

Also important is that _configuration is dynamic_. Although execution is decoupled from configuration, a new run triggers a fresh interpretation, 
effectively allowing a flow's configuration to be reconsidered before it's run again.

In [12]:
-- Avoid config-time error.
setEnv "SECOND_GREETING" "now this works!"
isJust <$> lookupEnv "SECOND_GREETING"

True

In [13]:
-- Now the previously config-time error is resolved.
runWithEmptyInput [goodFileArg, Arg $ ConfigFromEnv "SECOND_GREETING"]

Found docker images, pulling...
Pulling docker image: alpine:latest
2021-05-27T03:38:25.549357335Z Hello from the flow.yaml now this works!
Item {itemHash = ContentHash "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"}